# Baseline: Greedy Placement + Squeeze + Rotation Optimization

This notebook implements:
1. Greedy placement with weighted random angles
2. Squeeze/compaction to reduce bounding box
3. Rotation optimization (fix_direction) as post-processing

In [1]:
import math
import random
from decimal import Decimal, getcontext
import numpy as np
import pandas as pd
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

print('Imports done')

Imports done


In [2]:
# Tree geometry constants
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))
    
    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print('ChristmasTree class defined')

ChristmasTree class defined


In [3]:
def generate_weighted_angle():
    """Generates a random angle weighted by abs(sin(2*angle)) for better corner packing."""
    while True:
        angle = random.uniform(0, 2 * math.pi)
        if random.uniform(0, 1) < abs(math.sin(2 * angle)):
            return angle

def has_collision(candidate_poly, placed_polygons, tree_index):
    """Check if candidate polygon collides with any placed polygon."""
    possible_indices = tree_index.query(candidate_poly)
    return any((candidate_poly.intersects(placed_polygons[i]) and not
                candidate_poly.touches(placed_polygons[i]))
               for i in possible_indices)

def get_bounding_box_side(trees):
    """Get the side length of the bounding square for a list of trees."""
    if not trees:
        return Decimal('0')
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    width = maxx - minx
    height = maxy - miny
    return max(width, height)

print('Helper functions defined')

Helper functions defined


In [4]:
def initialize_trees(num_trees, existing_trees=None, num_attempts=10):
    """Greedy placement of trees."""
    if num_trees == 0:
        return [], Decimal('0')

    if existing_trees is None:
        placed_trees = []
    else:
        placed_trees = list(existing_trees)

    num_to_add = num_trees - len(placed_trees)

    if num_to_add > 0:
        unplaced_trees = [
            ChristmasTree(angle=random.uniform(0, 360)) for _ in range(num_to_add)]
        if not placed_trees:
            placed_trees.append(unplaced_trees.pop(0))

        for tree_to_place in unplaced_trees:
            placed_polygons = [p.polygon for p in placed_trees]
            tree_index = STRtree(placed_polygons)

            best_px = None
            best_py = None
            min_radius = Decimal('Infinity')

            for _ in range(num_attempts):
                angle = generate_weighted_angle()
                vx = Decimal(str(math.cos(angle)))
                vy = Decimal(str(math.sin(angle)))

                radius = Decimal('20.0')
                step_in = Decimal('0.5')

                collision_found = False
                while radius >= 0:
                    px = radius * vx
                    py = radius * vy

                    candidate_poly = affinity.translate(
                        tree_to_place.polygon,
                        xoff=float(px * scale_factor),
                        yoff=float(py * scale_factor))

                    if has_collision(candidate_poly, placed_polygons, tree_index):
                        collision_found = True
                        break
                    radius -= step_in

                if collision_found:
                    step_out = Decimal('0.05')
                    while True:
                        radius += step_out
                        px = radius * vx
                        py = radius * vy

                        candidate_poly = affinity.translate(
                            tree_to_place.polygon,
                            xoff=float(px * scale_factor),
                            yoff=float(py * scale_factor))

                        if not has_collision(candidate_poly, placed_polygons, tree_index):
                            break
                else:
                    radius = Decimal('0')
                    px = Decimal('0')
                    py = Decimal('0')

                if radius < min_radius:
                    min_radius = radius
                    best_px = px
                    best_py = py

            tree_to_place.center_x = best_px
            tree_to_place.center_y = best_py
            tree_to_place.polygon = affinity.translate(
                tree_to_place.polygon,
                xoff=float(tree_to_place.center_x * scale_factor),
                yoff=float(tree_to_place.center_y * scale_factor),
            )
            placed_trees.append(tree_to_place)

    side_length = get_bounding_box_side(placed_trees)
    return placed_trees, side_length

print('initialize_trees defined')

initialize_trees defined


In [5]:
def has_any_overlap(trees):
    """Check if any trees overlap."""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def squeeze(trees, max_iterations=100):
    """Squeeze trees toward center to reduce bounding box."""
    if len(trees) <= 1:
        return trees
    
    # Calculate centroid
    cx = sum(float(t.center_x) for t in trees) / len(trees)
    cy = sum(float(t.center_y) for t in trees) / len(trees)
    
    scales = [0.9995, 0.999, 0.9985, 0.998, 0.997, 0.995, 0.99]
    
    for iteration in range(max_iterations):
        improved = False
        for scale in scales:
            # Create trial configuration
            trial_trees = []
            for t in trees:
                new_x = cx + (float(t.center_x) - cx) * scale
                new_y = cy + (float(t.center_y) - cy) * scale
                trial_trees.append(ChristmasTree(new_x, new_y, str(t.angle)))
            
            if not has_any_overlap(trial_trees):
                trees = trial_trees
                improved = True
                break
        
        if not improved:
            break
    
    return trees

print('squeeze defined')

squeeze defined


In [6]:
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box."""
    if not trees:
        return Decimal('0'), 0.0
    
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)
    
    hull_points = points_np[ConvexHull(points_np).vertices]
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun
    
    improvement = initial_side - found_side
    EPSILON = 1e-8
    
    if improvement > EPSILON:
        return Decimal(str(found_side)), found_angle_deg
    else:
        return Decimal(str(initial_side)), 0.0

def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]
    
    # Get bounding box center
    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0]) / float(scale_factor)
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    
    rotated_trees = []
    for i in range(len(trees)):
        new_angle = float(trees[i].angle) + angle_deg
        new_tree = ChristmasTree(rotated[i, 0], rotated[i, 1], new_angle)
        rotated_trees.append(new_tree)
    return rotated_trees

print('rotation optimization defined')

rotation optimization defined


In [7]:
def process_configuration(n, existing_trees=None):
    """Process a single n-tree configuration with greedy + squeeze + rotation."""
    # Step 1: Greedy placement
    trees, _ = initialize_trees(n, existing_trees, num_attempts=10)
    
    # Step 2: Squeeze
    trees = squeeze(trees, max_iterations=50)
    
    # Step 3: Rotation optimization
    best_side, best_angle = optimize_rotation(trees)
    if abs(best_angle) > 1e-9:
        trees = apply_rotation(trees, best_angle)
    
    # Verify no overlaps
    if has_any_overlap(trees):
        print(f'WARNING: n={n} has overlaps after processing!')
    
    side_length = get_bounding_box_side(trees)
    return trees, side_length

print('process_configuration defined')

process_configuration defined


In [8]:
# Build the index of the submission
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]

# Process all configurations
start_time = time.time()
all_trees = {}
all_sides = {}

current_trees = None
for n in range(1, 201):
    trees, side = process_configuration(n, current_trees)
    all_trees[n] = trees
    all_sides[n] = side
    current_trees = trees
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        print(f'n={n}: side={float(side):.6f}, elapsed={elapsed:.1f}s')

print(f'\nTotal time: {time.time() - start_time:.1f}s')

n=20: side=4.229846, elapsed=0.5s


n=40: side=5.994236, elapsed=1.5s


n=60: side=6.913359, elapsed=2.8s


n=80: side=7.875484, elapsed=4.5s


n=100: side=9.016168, elapsed=6.7s


n=120: side=9.699300, elapsed=9.2s


n=140: side=10.688667, elapsed=12.1s


n=160: side=11.245814, elapsed=15.5s


n=180: side=11.766262, elapsed=19.3s


n=200: side=12.872723, elapsed=23.5s

Total time: 23.5s


In [9]:
# Calculate total score
def calculate_score(sides_dict):
    score = 0
    for n, side in sides_dict.items():
        score += float(side) ** 2 / n
    return score

total_score = calculate_score(all_sides)
print(f'Total score: {total_score:.6f}')
print(f'Target: 68.922808')

Total score: 162.204802
Target: 68.922808


In [10]:
# Create submission dataframe
tree_data = []
for n in range(1, 201):
    for tree in all_trees[n]:
        tree_data.append([float(tree.center_x), float(tree.center_y), float(tree.angle)])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(index=index, columns=cols, data=tree_data).rename_axis('id')

for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)

# Add 's' prefix
for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

print(submission.head(20))
print(f'\nShape: {submission.shape}')

                x           y          deg
id                                        
001_0  s-0.168647  s-0.054121  s225.000001
002_0   s0.032266   s-0.08489  s261.979346
002_1  s-0.109416   s0.232212  s391.183943
003_0   s0.305993   s0.146673  s327.748108
003_1   s-0.04132   s0.147619  s456.952706
003_2   s0.161177  s-0.395237    s88.42049
004_0   s0.162729   s0.168043  s390.143415
004_1   s0.000957  s-0.139296  s519.348013
004_2   s0.575848  s-0.211392  s150.815797
004_3   s0.444899   s0.518901  s283.051003
005_0  s-0.006963   s0.011632  s413.953014
005_1  s-0.030895  s-0.334857  s543.157612
005_2   s0.524173  s-0.168735  s174.625397
005_3   s0.109551    s0.44654  s306.860603
005_4   s0.900039   s-0.56315  s221.744773
006_0   s0.072749  s-0.345082  s491.158863
006_1   s0.405335   s-0.44515  s620.363461
006_2   s0.366256   s0.132925  s251.831245
006_3   s-0.32556  s-0.135151  s384.066452
006_4   s0.834114   s0.412116  s298.950622

Shape: (20100, 3)


In [11]:
# Save submission
import os
os.makedirs('/home/submission', exist_ok=True)
submission.to_csv('/home/submission/submission.csv')
print('Saved to /home/submission/submission.csv')

# Also save to experiment folder
submission.to_csv('/home/code/experiments/001_baseline_greedy_squeeze_rotation/submission.csv')
print('Saved to experiment folder')

Saved to /home/submission/submission.csv
Saved to experiment folder


In [12]:
# Validate submission - check for overlaps
print('Validating submission...')
overlap_count = 0
for n in range(1, 201):
    if has_any_overlap(all_trees[n]):
        print(f'  n={n}: OVERLAP!')
        overlap_count += 1

if overlap_count == 0:
    print('All configurations valid - no overlaps!')
else:
    print(f'WARNING: {overlap_count} configurations have overlaps!')

print(f'\nFinal score: {total_score:.6f}')

Validating submission...


All configurations valid - no overlaps!

Final score: 162.204802


In [13]:
# Let's check the sample submission score
sample_df = pd.read_csv('/home/data/sample_submission.csv')
print(sample_df.head(20))

# Parse and calculate score
def load_trees_from_df(df, n):
    """Load trees for configuration n from dataframe."""
    prefix = f'{n:03d}_'
    group_data = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row['x'])[1:]  # Remove 's' prefix
        y = str(row['y'])[1:]
        deg = str(row['deg'])[1:]
        trees.append(ChristmasTree(x, y, deg))
    return trees

sample_sides = {}
for n in range(1, 201):
    trees = load_trees_from_df(sample_df, n)
    sample_sides[n] = get_bounding_box_side(trees)

sample_score = calculate_score(sample_sides)
print(f'\nSample submission score: {sample_score:.6f}')

       id           x           y     deg
0   001_0        s0.0        s0.0   s90.0
1   002_0        s0.0        s0.0   s90.0
2   002_1   s0.202736  s-0.511271   s90.0
3   003_0        s0.0        s0.0   s90.0
4   003_1   s0.202736  s-0.511271   s90.0
5   003_2     s0.5206   s0.177413  s180.0
6   004_0        s0.0        s0.0   s90.0
7   004_1   s0.202736  s-0.511271   s90.0
8   004_2     s0.5206   s0.177413  s180.0
9   004_3  s-0.818657  s-0.228694  s180.0
10  005_0        s0.0        s0.0   s90.0
11  005_1   s0.202736  s-0.511271   s90.0
12  005_2     s0.5206   s0.177413  s180.0
13  005_3  s-0.818657  s-0.228694  s180.0
14  005_4   s0.111852   s0.893022  s180.0
15  006_0        s0.0        s0.0   s90.0
16  006_1   s0.202736  s-0.511271   s90.0
17  006_2     s0.5206   s0.177413  s180.0
18  006_3  s-0.818657  s-0.228694  s180.0
19  006_4   s0.111852   s0.893022  s180.0



Sample submission score: 173.652299
